In [1]:
import json
import numpy as np
import pandas as pd
import math
import ast

In [44]:
# with open('dataset/business_LV.json') as fd:
#     df = json.load(fd)
# print(len(df))
df = pd.read_csv('dataset/business_LV.csv')
print(len(df))

3923


In [45]:
# ["business_id", "neighborhood", "stars", "review_count", 
#             "RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups", 
#             "RestaurantsReservations","DriveThru", "WiFi","BusinessParking", "categories"]

In [46]:
def variables(all_columns,mode):
    cols = all_columns
    cat_cols = ["neighborhood", "RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups","RestaurantsReservations","DriveThru", "WiFi","BusinessParking", "categories"]
    if mode == "cat":
        cols = ["neighborhood", "categories"]
    elif mode == "bool":
        cols = ["RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups","RestaurantsReservations","DriveThru", "WiFi","BusinessParking"]
    elif mode == "cont":
        cols = ['stars', 'review_count']
    print(cols)
    return cols

In [48]:
# def format_long_data_cat(df,cols_to_feat,time):
    #First 6 hours
cols_to_feat = []
new_df = df.copy()
new_df = new_df.sort_values(['business_id'])
id_list = sorted(list(set(list(new_df['business_id']))))
cols = variables(new_df.columns,"cat")

for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        for col in cols:
            if (col == 'categories'):
                options = ast.literal_eval(row[col])
                for option in options:
                    if (option != "Restaurants" and option not in cols_to_feat):
                        cols_to_feat.append(option)
            else:
                suffix = row[col]
                if (type(suffix) is float and math.isnan(suffix)):
                    suffix = "missing"
                else:
                    suffix = row[col]
                if (col+'_'+suffix) not in cols_to_feat:
                    cols_to_feat.append(col+'_'+suffix)
print(cols_to_feat)


['neighborhood', 'categories']
['neighborhood_The Strip', 'Steakhouses', 'Cajun/Creole', 'Seafood', 'Bars', 'Nightlife', 'American (New)', 'neighborhood_missing', 'Thai', 'neighborhood_Eastside', 'Buffets', 'neighborhood_University', 'Burgers', 'Korean', 'Fast Food', 'neighborhood_Spring Valley', 'Comfort Food', 'Specialty Food', 'Asian Fusion', 'Food', 'neighborhood_Westside', 'Dive Bars', 'Indian', 'Bar Crawl', 'American (Traditional)', 'Chinese', 'Mexican', 'neighborhood_South Summerlin', 'Japanese', 'neighborhood_Sunrise', 'Latin American', 'Salvadoran', 'neighborhood_Chinatown', 'Noodles', 'Vietnamese', 'neighborhood_Southeast', 'Hawaiian', 'South African', 'African', 'Ramen', 'neighborhood_Summerlin', 'Middle Eastern', 'Mediterranean', 'Lebanese', 'Moroccan', 'Breakfast & Brunch', 'Event Planning & Services', 'Lounges', 'Venues & Event Spaces', 'Casinos', 'Arts & Entertainment', 'Beer', 'Wine & Spirits', 'Music Venues', 'Pubs', 'Beer Bar', 'Diners', 'Sushi Bars', 'neighborhood_So

In [49]:
cat_arr = np.zeros((len(id_list),len(cols_to_feat)))
print(cat_arr.shape)
i = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        feature_vector = np.zeros((len(cols_to_feat)))
        j = 0
#         print(row)
        for col in cols:
            if (col == "categories"):
                options = ast.literal_eval(row[col])
                for option in options:
                    if (option != "Restaurants"):
                        feature_vector[cols_to_feat.index(option)] = 1
            else:
                suffix = row[col]
                if (type(suffix) is float and math.isnan(suffix)):
                    suffix = "missing"
                else:
                    suffix = row[col]
                feature_vector[cols_to_feat.index(col+'_'+suffix)] = 1
        cat_arr[i] = feature_vector
        i+=1
print(len(cols_to_feat))

(3923, 374)
374


In [50]:
cols = variables(new_df.columns,"bool")
for col in cols:
    cols_to_feat.append(col)
bool_arr = np.zeros((len(id_list), len(cols)))
i = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        feature_vector = np.zeros((len(cols)))
        j = 0
        for col in cols:
            if bool(row[col]):
                feature_vector[cols.index(col)] = 1
        bool_arr[i] = feature_vector
        i+=1
print(bool_arr.shape)
print(len(cols_to_feat))

['RestaurantsTakeOut', 'GoodForKids', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'DriveThru', 'WiFi', 'BusinessParking']
(3923, 7)
381


In [53]:
# cols= variables(new_df.columns, "cont")
li1 = []
c = "review_count"
# for c in cols:
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        li1.append(row[c])


In [54]:
print(len(cols_to_feat))
count_arr=np.zeros((len(id_list), 5))
temp = pd.Series(li1)
bins = list(pd.qcut(pd.Series(li1),5,duplicates='drop',retbins=True)[1])
print(bins)
for i in range(len(bins)-1):
    cols_to_feat.append(c+str(bins[i])+'_'+str(bins[i+1]))
j = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        review_count = row[c]
#         print(review_count)
        feature_vector = np.zeros(5)
        for i in range(len(bins)-1):
            if bins[i] <= review_count and bins[i+1] >= review_count:
                feature_vector[i] = 1
                break
        count_arr[j] = feature_vector
        j+=1
print(count_arr[0])
print(len(cols_to_feat))

381
[3.0, 14.0, 38.0, 95.0, 261.0, 6979.0]
[ 0.  0.  0.  0.  1.]
386


In [55]:
rating_arr = np.zeros((len(id_list), 1))
c = "stars"
cols_to_feat.append(c)
j = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        feature_vector = np.zeros(1)
        feature_vector[0] = float(row[c])/5.0
        rating_arr[j] = feature_vector
        j+=1
print(rating_arr[0])

[ 0.8]


In [56]:
complete_data = np.concatenate((cat_arr,bool_arr,count_arr,rating_arr),axis=1)
print(complete_data.shape)
print(len(cols_to_feat))


(3923, 387)
387


In [62]:
def getIDPath():
    return 'dataset/ids.npy'

def getFeaturesPath():
    return 'dataset/features.npy'

def getDictPath():
    return 'dataset/cols2Dict.npy'

In [63]:
np.save(getFeaturesPath(),complete_data)
np.save(getDictPath(),cols_to_feat)
np.save(getIDPath(),id_list)

In [4]:
import json
import numpy as np
with open('dataset/user_ratings_LV.json') as fd:
    user_ratings = json.load(fd)
# len(user_ratings)
id_list = list(np.load('dataset/ids.npy'))
user_list = []
for user, ratings in user_ratings.items():
    for company, rating in ratings.items():
        if company in id_list:
            user_list.append(user)
            break

len(user_list)

183223

In [10]:
print(len(user_ratings))
users = list(user_ratings.keys()).copy()
for user in users:
    if user not in user_list:
        print(user)
        user_ratings.pop(user, None)
print(len(user_ratings))

229876
Zph0svqZTlcFnrZLQnyFHw
ZHpsZoBPtewDY0mg_JJm9A
IFjzqFbx5Nw_tqDccTeKLg
zDEb3qtjBl1Tl7BJRwC3ng
unfOc9o_joHB3N0yBzgaMg
N3PP8wNshR058jF4c6dRUw
sLBxJ_vbojWu1D0g6a8w9Q
vcc2glVWUfgSSCu1mmFM2Q
oUgbohZU7KtZYYiiovwV8g
BUaiDmE8hNSt1tiOvWlnPQ
doKileWfoM41UnKf0CefbQ
e0VhLvU3iQxVWQNxeBvc4Q
JVYPzKwGRA1dRZnYMYporw
IiwOipL5k2hnlEO6kb4v-Q
mRC2--f_7fjX8YXOzKLIng
obhgwrJcmqDYUO9hz5_CdA
5aQQrcTnJ0JW9D2-KZrb0g
J3EXvaqEkiW1b6Vuk3uN1g
L4uxvvEuc146Qo8wRipp6Q
MqzeU3_ZsboXT_0gIbqqMQ
NLKfw9hFAFc-ceuADj1y1w
OV4cZ8wLW0-mX0jUHt34RQ
155URnYGGuSa-lvtWxhI-A
-UtUXQksl5355cARklmcwQ
fyFg7G6Suh-bIkYIgMYzDQ
rwKv0qGp81yJD4pd-2vGXg
8kh928IbKacnnnCUOufgxg
Ho9C--zR9bTUHHv3WHPrPA
pzyPY25yA7weXRBylTuYqA
w2pAuV-y19jkLv4ue0N7Kg
hc_jEX3OTD6GFPMjnObPmQ
wjp9W-LvY-2POGFnyU15Fw
1Svnbn8Yz7eVRUuLxAT0cQ
sYYnGiGB4d0LE7JF-uBfhg
V6adIdOQqRaFwxnRZO0XNQ
-8PRz3Ibu6KdBPWmfWgCSw
ElQkNx53jRjWqWY7feB7zA
B3htbJNQL733-zkcZUpT4w
lhSNnT3apHXXF5E1UIUILg
ZcmeoMghVgMg7kycgzM8Ow
RPwT872z_dmPLxmfpFe4nA
YWcA8zNZf9kyh-I15vPOjg
wCaBV2_P_cNOHiFoDVIWEQ
SfEu

jk0VQUCRZydvXGVfIBAjRg
to-RK4mHxrVSdX95SlDmPQ
mr6X0jYSc-83cpIfQXwxMw
0-eYz1DKBSQ-BpI1KbeE9g
hQaXwYY06_j-4N_foOrLeQ
R5eBUlPdIfgvDpW2l2F05A
_2h9MazCl88U4Bnry1jHFQ
svMaU1gzYrqYDG3b5e0T9g
bV1tLgLUi8uK7DQK8dDiag
vVaznFzOAuOUU4Am9JIUdg
1zdDxBVPQEZTp01z5R_bNg
Rs1zXzwpVPIbtZyDpUrtkA
KyAUkrNyyq0T0Lf8Oq1AXg
at-BaiaSvcD3EdgDiGzVlw
nUg1LVMl4JHcIdhhvUW2Fw
G90Xiu0c3qMU1sx7iuFO1Q
lhyjHLeS1a5FLRDC4CBMAQ
IA6ry2AWcAVrn9xS1fa2Rg
7Y-jvudHI3mV4zel3SYdRA
JEIp11fTSLJYr-uJU2LPqQ
Ezc6dTxXCQhO-Fa-vh0QaA
UdImXP8G8WEzxhxY5tQXDw
zOU38EUHgUtEI4ziZ8DB1w
wO6wu7dnb3f6WQT9qGusDQ
96EsN2ceeTz_y_ixiEGieg
6pF5sGtuNlO1DIZNWqR5Vg
53-4NfDSmQepB9bYGUPoWA
zxweL7a6bkGo_woWpGmUaA
b9RWesYbanc_YRsJMCEs7A
POUloODbTGqjEj9vYtpBsg
WgmBsSDXbF5Vvw_I2Wug3Q
5jxxQ6LtqPkBfcBc7oQK1g
Mesmr_7gOslwjTBzYPlvCw
e9sAU1T6OGGPmNndlTBI9g
pzYNZZ7VwrsCsSf81Cai_w
D4QR37dAhs54dQZwqAtCRw
VJfQACKieUAMowjceyecQg
UmVV-oUIz1ehJPINWS7reA
LAbmWn-Zu-sJY7XsBp_4Zg
yluqHcOcaS-30v9TvvSx5g
BeCU6MnXIaKJRVrdTSB6Zg
NZXSWFYF10LfLoHAjTtK5w
pj4yb93ydJUqGQjrajwdtQ
TLH9Oqwturs

3zv_mIuUjqrhMpZ5yWoHoQ
Ou1gMZCJW8t7C8vVTEkUAw
YWtrbaXB6hYNEMXowPm0qg
7VRCuxl9p-Y0ojb-2hu2kQ
GUtzpeTjx1bcA6FsikWnzg
a07JUPEt4DQEs0NSqjR_vA
Hy7Q_GPjukJRdi7QmqtbgA
4YPmBnXkR0KKVAB3JgtvEA
VSVJZfeVe_XFf9-cm442Rw
HyKYGGAF-d9gLpE1VgQSGA
VPnVPlSTP7UefXNMyVlw2A
EXeg50LUzMQVTi4Y5BNdYg
CbAgGTqMPuUGHhzxnbSSVw
tZGUMcM_9FvkPXqo77Lrsg
GZLac2KeK21D2ib7NWbMrg
xfq4Xlqzn-nTc3sDIaa54w
Fjwl06IfB8DO5aDVqYt7tw
940iiliB_o9B5_ALfVwPIA
htE6iRnxx8mgtshB4Aa4Wg
bWoPQc-wh8-rJuYIvJxhVQ
Sf5Mwj9Hve1hUbzFDYZWXg
aoq4ldCiwlNoFQ7rZbVHTw
h2IsXZockcbACESTbvQIjQ
ZPoy-dlyGCkfEX-VC1HM3A
OazmRXabMb97kqIfuQzIUw
FUu8rpEfoxCMuvuR7sdqSw
t6gPlKaWCthDEDux9Myx7w
Gb5P8F9TALU88_qfAbyxLw
-hWXNVnN_YCdMFSJODEBGQ
nF2xJKiE4BGtIXq07cFTvg
OHmzAGrBoW-2N0zDw4Kfrw
mtP7lGJd8z4MXzHYheuCrw
sYnvuuAXYTuldKjasW-Kkg
QbeIE6NeNKcby6ffoM1yRg
umPXSZG6BQAl6TqT1xyMuA
8HstQmgpsHdTQvMzAi_ZHA
OGOs3gLM4GqMDJtAHta2Gg
h6AOKeQ8JIIvjLT-Oxhemw
_nCvlBNDtTxXwY8czeakxw
qwqkmc672g-7TPINVAtkBQ
rcuI80m3l-LbVTLG4XYjUA
M_JLWncGj6YMLR-oyUU3BA
Wotd28IpijT4YTNYNDKwQg
TG6-BMhRvTN

u7uuoa6LUbQTsgZKb2a2ww
F6C98ogp6BjNZhAZXr7brA
Q9u28gIRD24KG7vUXFFbjQ
ZvSqgWMTN1U2LA3zHrtlQw
d6susYXWo1kekQi-bhlV1w
MeIb-tKpULX-I3GC18g26w
IhS0DzOkcuaBJ6OWuRfRSw
eMC1-1cjYzLN2TORM89_Sw
kv9Vg5FbJiFgcNQ_FOJ2Ng
aplf-xWMTsP1RzHjs-0MMw
dcGK9kW4LGKqjbDdsHZ_3A
Cn2mmB9wAPiB0COnYpWpIg
lvyk1LSQjo1De8EJdKqKLQ
vU4j_vbnbmKGvEKLaNInfg
kX0LS2gV2xpgELPYhRFLXg
Jxx3roMctEh8UB3HF9mOHA
wdo4syn7g-E9L-lUXoXqdg
34BL1W16swI2NNztO_8WDg
IkpcIJdb8nnDTPVoZM73tw
NTnDglcvjtDdzb-twp05lw
PF-xY-NraknZKdbW22-kVg
SIao2vHyX1qnMXAv43OjYQ
z6agkKTcqneVjdByzqNtGg
lnqe4RbKVEPsIps6QybWtw
qWzJIFcnMx9Tkkak3iSfnA
fYyXRmPuS_Gmxla_STCFeg
5x90lSektW8hCpepfAIrgw
VT5jPDEp4kSf9Bwz0HjsQw
UDYGpn3zw3ND_lqhThBWRg
U4MuKQUwoQqK8Rqte4N62A
zoVJFQ5PDrtdwSWBH376qg
X5Uwp4R7iu16yGol8BGZ8Q
VPO_yCRMMsgp9a-F5Jp07A
-0oBg8owTGPK1rA5cFHWTA
YpOI6L25mnjxrr4uZESR-g
31x4QY7S2DxRBCDZDcG0Ug
Ll_bwtiw_b3QWXPUzuFa6Q
5JJly4w8LRVnWYzYiiShBQ
-c1P9j-NxIm0L1KCLCF8Vg
Sr91na8OJDW56V-s2XD2BA
ksROmj9ETbuHEAZyWqpaEg
FTKM2tg2IQOxQ9l5J8iVWA
i0mrxSV4FIIADCTWn8E4BQ
W4K4xSdV6Kb

83RCcl7uZpZRF7FqhM_GiQ
rL4RCN3fZFyQ92QKKvIKcA
jZc6OSiBLt1BG2oxGPakRQ
Y6zLfO3f2-w88Xhopwituw
T2Al_55TjQVtV0KpTy09-w
UO3h9Sz5rGQXlVwRa8XiEw
CEg6o7ZHHPUPuhDuNcrGKw
XN209p2ThQMJ3CY2QIfKKA
5yf9i_lsIo0OURxdvecYmA
GD7jKPt5kKKGUBVHmOLrYw
zrk_oFE4DtkQAHqhSIPAKg
LcSjb4aECPGfxmAhqTKijA
QabaBQ1Z_ZLhz85We5KBJQ
KsWLGLLJJNM6I0Pn0In8wg
23DjmEqlCm4Sp2mGaRSGsA
4lWnLmNdW3mAOIMgxqqyPA
h6YspRkEn_zb4dgGZTGNug
PS9w_OA2Gml5c-cO3Y0N0Q
9qpJcbi_zjQT4FkncfGLRQ
LaWXdyWxPy-Pzt2ne6qv0g
s9oqwnLmsJ41n5n_L6riWA
LlPBRQ3KzW5datZIjiHrWg
lljBIEk9J7EjRdgulQY4rQ
7Xr0m77k9W8DBF0LiTz48Q
SwOIi_6J9_pu_JRQOCriUA
QaOGbARMK_7cL6CnnOZ_aw
e130Q1s1IppBX2hwo_CBNw
ec2XhbUgpGK0ZVHoTacN1g
dvCvPwluak3jYHEFqeN8JQ
XumsEYzJ3KvVWTwUSbaiAg
54DQvmL5u7LXcArCIEIjig
qgIFLgIZ34JLdUpsQ4aWww
0Ac7RbtssfSsq-PKkSFAUw
1eryy-wunPafjqiFm07rdw
yBMSFdoChQX-94dlFe-AlA
W6Z3DaMvGQB0YUbIaO575Q
m7q3NRmxmjE44RQcTpnvbA
UV7LKBm5EgWQ3IqG6TOEBw
2QvvEgSODd6nMcN73GcBUQ
6_nidMyB68GzR1XkKeFiUQ
zlZxMAJQXLTSuG1Tj5GOow
BCjDNuksFpWAOb5xBJJAEQ
CwBB9bNIaGXLdRA_nlOzvQ
Xm6FMQrjnHJ

...

GvoVAsQ9SW1ZnvicqT6Mcw
AABe1-sKO4cXySILVHQkAA
NZ4SD75oy-1JAY4zm_7N0w
u1eIWrC3735nQfA40dU-3w
_4YU80uUmdxmLkF-gozjmA
KmVV6oD1kRJlHzuX7lN-bw
NR4ZhmYktFXNhtBsklR41Q
5mdEwOvdO_SdSQF_jGriFg
rfb3T1JvRygUvH0U1d810A
TE2LnVJxXI76rP2V8knVyQ
VGhMkrtVEmPqPgdcl244nQ
uXVyIg5u3y18I-hr-cnj_A
YhG5UhmdlhnM89C6-kKRdQ
UMLgiIuf0vEqo6GatvzyOQ
hXzlYtXfY4stT3UobBsDUw
eTJdwwtvhbIBQDh56NmEkA
jBtMJSuLSVM9-Ji4epn4YA
qD4qwLDxywywLgwLj7Df2w
7YncVmyGSO4XUDAc029X5w
43bemeL_L1SPsVLlpzyuSg
cuj8Cx4573JKU4lz9mASfA
VWbkzKMiCrVeIKWAXENNBQ
wjADoBgiOx8cGBoLVJ5Wag
ldhYcwY-rIMqSnln_ybcyg
nti9x6FZTSCn27u-pYTMig
To4sKy81uFPUr-8OQorB5g
VSw3PKa_f2OwoAZyFasbbg
t3hEx-LIAhmPGp0I6kXI4A
UnwgqzPS5hPrTp0ct-UxAg
YAMXTP4K0sG-9uHjYjzRDQ
xev35uk-mesNA8MhisYIRA
uRmtk0O6m72NVseg6OqeRw
6RceFpguzisxPA0A7BwxLA
prQbzlN3DA-gMvGkiLDxPA
88ReOeLvgpYDuUfcNTDKuw
DBt0OKgKeYONrefL_Oi7dQ
pWoKEgjw5UO-EFcdgL0H4A
vdYErSpsfizAk69_tW6-ZQ
4MlfdJ820soBR-uikxAGng
O7xMYG03v3FAKc2ha_uNAQ
3aw2Di_RZeCU3LBWZY8iYQ
fzHt5LPBuwyNc1F-5aMDtw
U-sm7WYQzK9VXK-mNmctNg
NItZ1WmeLTh

2jFaJD5o22zT8IjcIR9DEA
yUgdeYi9gOEDf7N_aJEjzg
yW2lfR7-2xVvTumKsG866w
mPDLqRh_AQ9W_axuM3eRAg
mkor_zCQOF5B7HFVIhxa3Q
MoRzN09ibKg_0wV2MnIw1A
Z6cuROCsyyVeQU3HwV44CQ
dPE42F-I_FvPvoUxYhqujQ
5Ui8oFp_nd5kMHVe2Itekg
VfGUWv5Pek_VVRyZUafOCA
ZdMGiKlAY8uHmRRh-rT9DQ
COxMWO2c5nP1tydSxv1TZg
6KTOKGPQp0lkpvrn-W7D4Q
EzkJp21qT_bat0V6AApJyQ
uHL4VEryxfL5fnWABeCltg
yjUwtqAygWaOahxEW-6I1A
kIJSRuJNXvjCGouiO7K2ZQ
Qb23hua-gH7fPcCCj4Fg-g
wmkZf8ok_iXfasVhKUcpWg
rkaRvKdt9MCj0wnayiB1Hg
IFap8kfFQeUkgHX66UGrpQ
1AwBUK4nENCDFHAZo5c6OQ
Pxe1XCkhSHgyARpN5jDpQg
up0HZlyEJT8sVoAD8fFA8g
JxE5fY4_3XVSgU_x8Q18lA
G9K6nBQ4UF51iNEFodvF6w
PSCOl3WQ3C3yM-qbX0aa7A
fOHAzCVvan3Bm0SpG1rEeA
TE3wzd1egJmpvMDK7XnKZw
XMW8gS3V-52slMap1PYz1g
rZVXenaW808-BAF5EcWcMA
rIS3DPN8ti51SVY2zlVyVQ
mLOM_AXCpOKJLDg_DfTJ5A
GiP3O8ZeZB-SEubVF7xtsQ
vkzUXEJRma8nGPuRkq1VeA
yp_wJGRnxShGymmmAIeqCg
fztHYg_kktq6gWOi-hNEvw
_N0lvG5LTNprzdGIZ-rZ2g
jir39t72bZxSXqXByMGFbA
7h0SReQXwL2qqpWFkPDufg
88X8HzeexdWHuzfaw3e4Kg
UKDyTALcQWyD0wZjBS3E2Q
jBnP54g6pD2MsbXRFdABCA
8FKitNzklLK

XGE3yRMDwghKsL-WJXk1ww
4qyK5u4GDPPn-aYn_KusgQ
3VVpT6wy1OOkr_4QDx7b2Q
JS4FVqOWbfhonFnegx5cPA
iYq6DulDccjqMIkzQOtRkw
GTaGhZXpETnj1wuK7hPnbw
p6lRDAV_46H0F5oAa__n0w
fdVDrjcYJRxnIeqRoGvuFg
0Ro1r6wD8O5nVAZpJPnoGw
aGVQHHM3R-k0JNOt4026yw
2vFUeQGnG8zdLSQkwfChog
yZbDsPetl-ePLTxywjU58A
tTwpzagdZ681E5JUIPaDig
1gzI5O_zcj2_FtuQrQILnw
BFIaTEKz0kUoo9WZNldPJQ
NqBhlrvDVOPHyJUUY256OA
Dm5zd2Ri4hdgSZgwRqQlYw
LuWlBxP6-DMFpJbYA5zWPw
xuLFeSwSYWAtIIV3WmmALQ
YnH5z31wQFcR4oZ3xRYkKQ
tXUf1M2ckC3C7lQwv2Z3tA
DD5hv2gZ1bnInE107SpfOg
NFgKnLQomlrNueFchUO9Mg
WDCoz59Xz1_ys8KjAohUFA
3ge0aBKJgUio-aGrsIbSrw
hSwMiEGne1POa4wNzNLz6g
KEskDUDL1JRiz08Nc0NGkQ
y-J0p4-5Ibk8Qj9pUMMcmg
yAAsncTlazgKk-WW0XVBCQ
bQkE6hKG0tLnRk6ZILLM_g
Klgv-6-EW2KaLeHoSylGGA
KwRH-AC-R077WT4pfkiYLw
bwpIRk5UEEcAemzHXL_QVw
Qlsz9NSll7r6CPWqAoNMKA
jALMMuZ4s3Meg3Uq0NzLVw
e5mIJsH30hehgaJaGt7LfQ
NxP16Jw8oS8BkV4FQqD8DA
RvIVIyphElTeE0yMpfF00w
38RefagdxAvHUmRL33ak-Q
gXJEpCuBnivbVdH_vI9rrw
t88a5dmrDU7dnX1LqvW3sQ
_fEmpHgp6VQVHfmPqwlgkg
obIPhHuaE4SMfgpca5qKvQ
widVEsQzo4H

svSPWzlwe5hUdjsudmucSg
rJQ8Px8VdzXg2-P8FtK2DQ
LGzP24xLVFNyjaxiLMynJA
SsxO4EJJUBleXPi-ly16Ew
f3JJczzAPLSbDdHmYEjKTg
CWyAQKabbrGt_456iJJNnQ
or12r_LO_XDi2QIulbol4g
FX-mpFj0OpC0wLei5LA7XQ
Bt5n-UC8aRVqKvTtQsJ7kA
wJ1KEqvUASdFfis-KsVDxA
90XesmLDW1Y2XoSz1bUlxQ
6hzCuCPif-fQcTptPEfiew
WhPlUwHQfg7I5DvauKTimQ
KbZD54Pi-TWSjw2I5V6dug
apsTpVccu3tgWDsTR0EDfQ
_KCBn74huAKJQrVDrvgw0A
FnnpYFfUjrdJQVTTlUXynA
f310XqHZk1vJGC5egrR8Mg
f0Qpk9IWVAM80klBBGBRlQ
FS57B_wj14pcHU50sV4cMA
ukG9BtQnNiBE8R-YUyHUmw
KuK_BG10-NY87v1KWW9SWQ
qwOpugpo_diLGImsHVd57Q
zcmCZJ0IIjWoYausynM2Gw
A4eiKH-QnNR9T1x0AsruRw
_ZnPZlMdrvDgZmeKK4XqiA
tNVZQLQFLb6HvrwRCWAhdg
9MVKS8tgtEHRVsk8_2zXxA
TE3Ie2hM1EEfLixNq1Idmw
kjSbcIfWY8Oamx9m01EwhQ
J3DrZw73e79H2KncokX6Tw
CJb8CEyI6MAOOnDANe9u8w
OwfIVv5ddbOxHCkpuFX8sA
m6pBShtTyFTxina6CmJdHg
QAEajAHvLQNjDHgZMFjEAw
UUCxlgfpbXhGpvfe57cWYA
7i7if9g8VKHVcYFD_ntAHg
HlJr9yKRNAbzkc-p6F8_cg
efpS2alT7HYQvNtupte5Eg
sCoLK0PmulKaE8Ra8VUAJw
mVL7H7y84tedV-pTVeMLdA
FS1ZoS-3t-pcJr4NXg13Yw
rTYvqV35g0Te7QJIVKC42g
Sk3utvuKU8l

Fo0pR-IzTo359_SiQ3wk2A
qbAZkU-gec0m47omqm7HqA
Rc4fopwWmjDgFxMPfr8L_g
DkFfjGCTJgR1N0qI2fikUw
JRRmM3St3-k_dPge3vOZfQ
co1eCvQ_wTVwR5E26wgWXQ
SxA4Bn-GoUnhkCUWRwMeFg
aNtMb7qLr_o-lBRP7j6C-g
y1NpbtmQWmTcbPi4ivZNsg
QmZOw-h8GtsDqmHxJXJ1cw
q8ZmFnFXoK6x89pSmzqxVA
upBistIfRJ-hEJlDTS_22A
MBDFQLEB8kCmHq4S2jU_ng
XNG3V6r561MdVKMkDB08Hw
h86ON_v2jVFxBwyFn1qKEg
RwFTH7kcWeGyir9Xt81iDQ
_uEn3cxgJ0eDt8T_JzC3Wg
a4lPSFqBtcr_Gu0pWA2fAg
wXNvvrY-to1k8uD1O2HR7A
A0di6na2UuDrIJXSnm_b9g
bIeMIr0lwkqrumKyUn3nqQ
HrUMfoQ5Gz0PubfNsz-43A
tUsyy4Ot8sWaOt9Iq-QPUw
FC346bJHb1u5eTwpmTlO_w
YOS_zRMC_6rkGyaIH1zgag
BIIcheAmBX0Pe15xJO8efA
uzME9zLJs6YXf7SLdX732g
ov_2zFRBO_ixL9FMnKimgA
4po0_r_C5UO_e5Awu53WiA
ROCeGUQi0nHpeCa-d5Ld8g
bK5zlFcqhAgozNfJq4uGaw
FapnD0gDefRJH9vdF1fVaw
yX-_iWKwNtfO_UoTgsVxwQ
m029zFt91exq-gYNQ9ZQTw
5GARyRIOCmB9firUPqlxog
2wABjv6o-sa4saKeCi2Pqg
keL5PqaIpwe1lSmP6atgmQ
btTsWgT4nYnnMPj8ru6W_g
BZinfFX4rzkrIXV933nyRA
7ds2_rjoXT9nbrzZgY7fbA
lnwbeGVRd4Gcy73LpqXBNg
qNpRsHJ5hQI3DO_a7xlsRg
Xh1SJ2Cju2BGvRe7IGZHrg
uvjtTdUS3Df

In [11]:
with open('dataset/user_ratings_LV_filtered.json', 'w+') as fd:
    json.dump(user_ratings,fd)

In [12]:
with open('dataset/user_directory_LV_filtered.json', 'w+') as fd:
    json.dump(user_list,fd)

In [13]:
with open('dataset/user_ratings_LV_filtered.json') as fd:
    temp = json.load(fd)
print(len(temp))

183223
